# PACEUTILS

A package for computing values and data frames from the PACE-RI database. This guide can also be used to define certain calculations. Once the files are downloaded it can be installed using the command below (any updates by the data analyst to paceutils files will need be copied into the folder for the install to be updated);

`python -m pip install -e c:\path_to_paceutils_folder`

### Importing All Modules

In [1]:
import paceutils as pu

### Importing A Single Module

In [2]:
from paceutils import Enrollment

### Specifying the Database
The connection is opened and closed in each function.

In [3]:
e = Enrollment() #default V:\Databases\PaceDashboard.db
# or
e = Enrollment("V:\\Databases\\PaceDashboard.db")

### Running Functions
You can run from the variable the class is initialized to or from the class itself.

In [4]:
e.census_during_period(params = ('2019-01-01', '2019-01-01'))

307

In [5]:
Enrollment().census_during_period(params = ('2019-01-01', '2019-01-01'))

307

### Common Definitions for Calculations (some of these are reiterated below)
**Member Months:** A participant is included in the member month count if they are enrolled as of the first of the month. This means a ppt enrolled on July 1st and disenrolled on September 22nd would have a member month count of 3.

**Net Enrollment:** The difference between the new enrollments on the first of a month and the disenrollments from the previous month.

**Census as of Date:** Count of ppts who have an enrollment date before or on the date and a disenrollment date on or after the date.

**Census Today:** Count of ppts with an enrollment date before or on today and no disenrollment date.

**Census Over Period:** Count of participants with a disenrollment date on or after the start date and an enrollment date before or on the end date of the period.

**Average Years Enrolled:** The difference between today or the ppts disenrollment date and their enrollment date divided by 365.25.

**Growth:** The difference between the census at the end of a period and the census at the start of a period divided by the census at the start of the period.

**Churn:** The quotient of disenrollments during a period and the census on the start date of the period.

**Adjusted Incident Count:** The count of an incidents minus any incidents attributed to ppts with an incident count 3 standard deviations above the average for ppts with an incident. I.e.: If the average number of falls per faller is 2 with a standard deviation of 0.5, then the falls of any ppts with a count above 2+(0.5\*3) = 3.5 would be excluded from the adjusted count.

**Living in the Community:** Count of ppts not in custodial care. Included respite, skilled, and assisted living ppts.

**Days vs LOS**: Length of Stay is only calculated for discharges and will cover the entire stay. Days is calculated for all admissions and if there is no discharge date, the end date of the period is used. When counting days in a month, the first of the month is used as the admission date for any admission with a date before the period.

## General Use Helpers

Functions for querying the database and parsing dates. ***All*** other classes inherit from the Helpers class.

In [6]:
helpers = pu.Helpers()

### Single Value Query

Returns a single value from the database, uses .fetchone from the sqlite3 package. Takes a query and any necessary parameters (note: params can be left blank).

In [7]:
helpers.single_value_query(q = "SELECT total FROM monthly_census WHERE month = ?",
                           params = ('2019-01-01',))

307

In [8]:
helpers.single_value_query(q = "SELECT total FROM monthly_census WHERE month = '2019-01-01'")

307

### List of Tuple Query

Returns a list of tuples from the database, uses .fetchall from the sqlite3 package. Takes a query and any necessary parameters (note: params can be left blank).

In [9]:
helpers.fetchall_query(q = "SELECT * FROM monthly_census WHERE MONTH >= ? LIMIT 5",
                       params = ('2019-01-01',))

[('2019-01-01', 242, 58, 7, 307),
 ('2019-02-01', 243, 59, 7, 309),
 ('2019-03-01', 242, 60, 7, 309),
 ('2019-04-01', 245, 65, 8, 318),
 ('2019-05-01', 241, 66, 9, 316)]

In [10]:
helpers.fetchall_query(q = "SELECT * FROM monthly_census WHERE MONTH >= '2019-01-01' LIMIT 5")

[('2019-01-01', 242, 58, 7, 307),
 ('2019-02-01', 243, 59, 7, 309),
 ('2019-03-01', 242, 60, 7, 309),
 ('2019-04-01', 245, 65, 8, 318),
 ('2019-05-01', 241, 66, 9, 316)]

### DataFrame Query

Returns a pandas DataFrame from the database, uses pd.read_sql. Takes a query and any necessary parameters (note: params can be left blank).

In [11]:
helpers.dataframe_query(q = "SELECT * FROM monthly_census WHERE MONTH >= ? LIMIT 5",
                        params = ('2019-01-01',))

,month,providence,woonsocket,westerly,total
0,2019-01-01,242,58,7,307
1,2019-02-01,243,59,7,309
2,2019-03-01,242,60,7,309
3,2019-04-01,245,65,8,318
4,2019-05-01,241,66,9,316


In [12]:
helpers.dataframe_query(q = "SELECT * FROM monthly_census WHERE MONTH >= '2019-01-01' LIMIT 5")

,month,providence,woonsocket,westerly,total
0,2019-01-01,242,58,7,307
1,2019-02-01,243,59,7,309
2,2019-03-01,242,60,7,309
3,2019-04-01,245,65,8,318
4,2019-05-01,241,66,9,316


### Create a Time/Value DataFrame for Plotting

Takes an indicator calculating function and creates a pandas DataFrame of the monthly or quarterly value (based on freq parameter) for this indicator over the provided params. If no params are provided, it will create a DataFrame for the previous year. if the indicator need additional parameters other than date ranges, they can be provided in additional_func_args.

In [13]:
helpers.loop_plot_df(indicator_func = e.census_on_end_date,
                     params = ('2019-01-01', '2019-09-30'),
                     freq="MS",
                     additional_func_args=None)

,Month,Value
0,2019-01-01,304
1,2019-02-01,305
2,2019-03-01,306
3,2019-04-01,313
4,2019-05-01,312
5,2019-06-01,315
6,2019-07-01,319
7,2019-08-01,322
8,2019-09-01,336


In [14]:
helpers.loop_plot_df(indicator_func = e.census_on_end_date, freq="QS")

,Month,Value
0,2019-01-01,306
1,2019-04-01,315
2,2019-07-01,336
3,2019-10-01,339
4,2020-01-01,346


### Date-Related Helper Functions
The following require no parameters and return the dates based on today.

In [15]:
print(helpers.month_to_date())
print(helpers.last_month())
print(helpers.last_three_months())
print(helpers.last_six_months())
print(helpers.last_year())
print(helpers.last_quarter())
print(helpers.quarter_to_date())

('2020-02-01', '2020-02-18')
('2020-01-01', '2020-01-31')
('2019-11-01', '2020-01-31')
('2019-08-01', '2020-01-31')
('2019-02-01', '2020-01-31')
('2019-10-01', '2019-12-31')
('2020-01-01', '2020-03-31')


Last quarter can also the number and year of the previous quarter.

In [16]:
helpers.last_quarter(return_q=True)

(4, 2019)

Returns the date for a given quarter and year.

In [17]:
helpers.get_quarter_dates(q=2, yr=2019)

('2019-04-01', '2019-06-30')

Takes a tuple of dates as the parameters and returns the dates for either the previous month or quarter.

In [18]:
helpers.prev_month_dates(params=helpers.last_month())

('2019-12-01', '2019-12-31')

In [19]:
helpers.prev_quarter_dates(params=helpers.last_quarter())

('2019-07-01', '2019-09-30')

## Enrollment

In [20]:
enrollment = Enrollment()

### Census Today
Count of participants in enrollment file with a null disenrollment date.

In [21]:
enrollment.census_today()

346

In [22]:
params=("2019-01-01", "2019-03-31")

### Census During A Period
Count of ppts with an enrollment date prior to the end date and a disenrollment date that is null or after the start date.

In [23]:
enrollment.census_during_period(params=params)

320

### Census at the End of a Period
Count of ppts with an enrollment date before the start date and a disenrollment date that is null or after the end date.

In [24]:
enrollment.census_on_end_date(params=params)

306

### Member Months
Sum of the first of the month census for each month between the two param dates.

In [25]:
enrollment.member_months(params=params)

925

### Disenrolled
Count of ppts with a disenrollment date between the two param dates.

In [26]:
enrollment.disenrolled(params=params)

15

### Enrolled
Count of ppts with a enrollment date between the two param dates.

In [27]:
enrollment.enrolled(params=params)

19

### Deaths
Count of ppts with a disenrollment date between the two param dates and a disenroll type of 'Deceased'.

In [28]:
enrollment.deaths(params=params)

10

### Net Enrollment (total during period)
`enrolled` - `disenrolled` for the dates between the two param dates.

In [29]:
enrollment.net_enrollment_during_period(params=params)

4

### Net Enrollment (for month)
`enrolled` for the dates between the two param dates - `disenrolled` for the month before the starting date of the params.

Difference between ppts enrolled on the first of a month and the disenrollments of the previous month. For a quarter this means we are adding up the number of new ppts on the first of each month in quarter and subtracting the number of disenrollment from the month before the quarter through the middle month of the quarter.

In [30]:
enrollment.net_enrollment(params=params)

5

### Voluntary Disenrollments
Count of ppts with a disenrollment date between the two param dates and a disenroll type of 'Voluntary'.

In [31]:
enrollment.voluntary_disenrolled(params=params)

5

### Percent of Disenrollments that are Voluntary
`voluntary_disenrolled` divided by `disenrolled` multiplied by 100.

In [32]:
enrollment.voluntary_disenrolled_percent(params=params)

33.33

### Average Years Enrolled in PACE (for ppts enrolled in period)
Average of the difference between today's date or the ppt's disenrollment date and their enrollment date divided by 326.25.

In [33]:
enrollment.avg_years_enrolled(params=params)

4.4

### Growth Rate
Census as of the first of the previous month from the given param dates minus the census as of the first of the month for the given params divided bu the census as of the first of the previous month from the given param dates multiplied by 100.

In [34]:
enrollment.growth_rate(params=params)

0.99

### Churn Rate
`disenrolled` for the given params divided by the census as of the first date in params tuple.

In [35]:
enrollment.churn_rate(params=params)

4.89

### Count of Ppts per Town (for ppts enrolled in period)
Count of distinct ppts grouped by town for ppts enrolled during the period. Returned as a pandas dataframe with columns `City/Town` and `Number of Ppts`.

In [36]:
enrollment.enrollment_by_town_table(params=params).head()

,City/Town,Number of Ppts
9,Providence,163
11,Pawtucket,28
24,Cranston,26
0,Woonsocket,23
3,Warwick,22


### Mapping DataFrame
Creates two pandas dataframe with the columns `name`, `full_address`, `lat`, and `lon`, one for ppts enrolled during the period and one for those who are not.

In [37]:
enrolled_df, disenrolled_df = enrollment.address_mapping_df()

print(enrolled_df.head())

                  name                        full_address        lat  \
0           Mary Ricci  10 Rhodes Avenue, North Smithfield  42.002005   
1   M'liss Place-Sloan            568 Child Street, Warren  41.732102   
2        Linda Killeen             25 Tobey St, Providence  41.819837   
3           Sida Krong         100 Atwells Ave, Providence  41.822834   
4          Maria Alves           100 Borden St, Providence  41.812769   

         lon  
0 -71.538138  
1 -71.254934  
2 -71.436050  
3 -71.421123  
4 -71.411593  


### HPMS Related

### Dual Enrolled Enrollments
Count of ppts with an enrollment date during the period, medicare status of 1 and a medicaid status of 1.

In [38]:
enrollment.dual_enrolled(params=params)

19

### Medicare Only Enrollments
Count of ppts with an enrollment date during the period, medicare status of 1 and a medicaid status of 0.

In [39]:
enrollment.medicare_only_enrolled(params=params)

0

### medicaid Only Enrollments
Count of ppts with an enrollment date during the period, medicare status of 0 and a medicaid status of 1.

In [40]:
enrollment.medicaid_only_enrolled(params=params)

0

### Private Pay Enrollments
Count of ppts with an enrollment date during the period, medicare status of 0 and a medicaid status of 0.

In [41]:
enrollment.private_pay_enrolled(params=params)

0

### Dual Enrolled Disenrollments
Count of ppts with an disenrollment date during the period, medicare status of 1 and a medicaid status of 1.

In [42]:
enrollment.dual_disenrolled(params=params)

15

### Medicare Only Disenrollments
Count of ppts with an disenrollment date during the period, medicare status of 1 and a medicaid status of 0.

In [43]:
enrollment.medicare_only_disenrolled(params=params)

0

### medicaid Only Disenrollments
Count of ppts with an disenrollment date during the period, medicare status of 0 and a medicaid status of 1.

In [44]:
enrollment.medicaid_only_disenrolled(params=params)

0

### Private Pay Disenrollments
Count of ppts with an disenrollment date during the period, medicare status of 0 and a medicaid status of 0.

In [45]:
enrollment.private_pay_disenrolled(params=params)

0

### Marketing/Enrollment Referral Related

### Inquiries/Referrals
Count of rows from the referral table where the referral_date is between the param dates.

In [46]:
enrollment.inquiries(params=params)

116

### Average Days to Enrollment
Average difference of the enrollment_effective date and the referral date for referrals with an enrollment_effective date between the param dates.

In [47]:
enrollment.avg_days_to_enrollment(params=params)

114.56

### 180 Day Enrollment Conversion Rate for Referrals
Count of all referrals with a referral date between 180 days before the param start date and the param end date with a non-null enrollment effective date divided by the count of all referrals with a referral date between 180 days before the param start date and the param end date.

In [48]:
enrollment.conversion_rate_180_days(params=params)

0.26

### Count of Referrals by Source
Count of referrals grouped by source for referrals with a referral date during the period. Returned as a pandas dataframe with columns `referral_source` and `referrals`.

In [49]:
enrollment.referral_source_count(params=params).head()

,referral_source,referrals
0,TV/Ad/Media,16
1,Blackstone Valley Community Health Center,15
2,Thundermist-Woonsocket,8
3,PACE participant,8
4,Thundermist-West Warwick,7


### Most Common Referral Source
Creates the `referral_source_count` table and returns the first row as a tuple.

In [50]:
enrollment.most_common_referral_source(params=params)

('TV/Ad/Media', 16)

### Enrollment Conversion Rate by Referral Source
Count of referrals with a non-null enrollment_effective date divided by the count of referrals grouped by source for referrals with a referral date during the period. Returned as a pandas dataframe with columns `referral_source` and `enrollment_rate`.

In [51]:
enrollment.referral_enrollment_rates_df(params=params).head()

,referral_source,enrollment_rate
0,The Highlands,1.0
1,Sisters Of Mercy,1.0
2,PACE Van,1.0
3,Neighborhood Health Plan,1.0
4,Lifespan Primary Care Clinic,1.0


### Source with the Highest Enrollment Conversion Rate
Creates the `referral_enrollment_rates_df` table and returns the first row as a tuple.

In [52]:
enrollment.highest_enrollment_rate_referral_source(params=params)

('The Highlands', 1.0)

## Demographics

In [53]:
demographics = pu.Demographics()

### Number of Dual Enrolled Ppts
Count of ppts enrolled in the period with a medicare value of 1 and a medicaid value of 1.

In [54]:
demographics.dual_count(params=params)

301

### Percent of Ppts that are Dual Enrolled 
`dual_count` divided by `census_during_period`.

In [55]:
demographics.percent_dual(params=params)

94.0

### Number of Medicare Only Ppts
Count of ppts enrolled in the period with a medicare value of 1 and a medicaid value of 0.

In [56]:
demographics.medicare_only_count(params=params)

1

### Percent of Ppts that are Medicare Only
`medicare_only_count` divided by `census_during_period`.

In [57]:
demographics.percent_medicare_only(params=params)

0.0

### Number of medicaid Only Ppts
Count of ppts enrolled in the period with a medicare value of 0 and a medicaid value of 1.

In [58]:
demographics.medicaid_only_count(params=params)

18

### Percent of Ppts that are medicaid Only

`medicaid_only_count` divided by `census_during_period`.

In [59]:
demographics.percent_medicaid_only(params=params)

6.0

### Number of Private Pay Ppts
Count of ppts enrolled in the period with a medicare value of 0 and a medicaid value of 0.

In [60]:
demographics.private_pay_count(params=params)

0

### Percent of Ppts that are Private Pay
`private_pay_count` divided by `census_during_period`.

In [61]:
demographics.percent_private_pay(params=params)

0.0

### Average Age
Age is calculated as the difference between the end date and the ppt's date of birth.
Average of age for ppts enrolled in the period.

In [62]:
demographics.avg_age(params=params)

76.34

### Number of Ppts Below Age 65
Count of ppts with an age below 65 as of the end date in the params.

In [63]:
demographics.age_below_65(params=params)

45

### Percent of Ppts Below Age 65
`age_below_65` divided by `census_during_period`.

In [64]:
demographics.percent_age_below_65(params=params)

14.06

### Number of Ppts Above Age 65
Difference between `census_during_period` and `age_below_65`.

In [65]:
demographics.age_above_65(params=params)

275

### Percent Ppts with a Primary Language that is not English
Count of rows in demographic table where the language is 'English' divided by the count of ppts enrolled in the period.

In [66]:
demographics.percent_primary_non_english(params=params)

39.69

### Percent of Ppts with a race that is not Caucasian/White
Count of rows in demographic table where the race is 'Caucasian/White' divided by the count of ppts enrolled in the period.

In [67]:
demographics.percent_non_white(params=params)

50.31

### Number of Female Ppts
Count of ppts in the demographics table where the gender is 1 and the ppt is enrolled.

In [68]:
demographics.female_count(params=params)

222

### Percent of Ppts that are Female
`female_count` divided by `census_during_period`

In [69]:
demographics.percent_female(params=params)

69.38

### Number of Ppts with a Behavioral Health Diagnosis
Count of distinct ppts from dx table where the ICD10 code is in F2, F31, F32, F33, F4, or F6 and the ppt is enrolled during the period.

In [70]:
demographics.behavorial_dx_count(params=params)

252

### Percent of Ppts with a Behavioral Health Diagnosis
`behavorial_dx_count` divided by `census_during_period`

In [71]:
demographics.behavorial_dx_percent(params=params)

78.75

### Number of Ppts with a Dementia Related Diagnosis
Count of distinct ppts from dx table where the ICD10 code is in F01.50, F01.51, F02.80, F02.81, F03.90, F03.91, F10.27, F10.97, 
F13.27, F13.97, F18.17, F18.27, F19.27, F19.97, G31.09, G31.83, G30.00, G30.10, G30.08, G30.09 and the ppt is enrolled during the period.

In [72]:
demographics.dementia_dx_count(params=params)

130

### Percent of Ppts with a Dementia Related Diagnosis
Count of distinct ppts from dx table where the ICD10 code is in the chronic condition list (see code for full list) and the ppt is enrolled during the period.

In [73]:
demographics.at_least_one_chronic_condition_count(params=params)

318

### Count of Chronic Conditions per Ppt

Counts the number of distinct icd10 codes in the chronic condition list grouped by ppt enrolled during the period. Returns a dataframe with the columns `member_id` and `count`.

In [74]:
demographics.chronic_condition_df(params=params).head()

,member_id,count
0,100113,7
1,100116,4
2,100121,15
3,100128,4
4,100131,5


### Number of Ppts with more than 6 Chronic Conditions
Creates the dataframe from `chronic_condition_df` and reutnrs the count of those rows with a count above 6.

In [75]:
demographics.over_six_chronic_conditions_count(params=params)

163

### Percent of Ppts with more than 6 Chronic Conditions
`over_six_chronic_conditions_count` divided by `census_during_period`

In [76]:
demographics.over_six_chronic_conditions_percent(params=params)

50.94

### Number of Ppts Living in the Community
Count of ppts enrolled during the period who do not have a custodial admission date during the period.

In [77]:
demographics.living_in_community(params=params)

285

### Percent of Ppts Living in the Community
`living_in_community` divided by `census_during_period`.

In [78]:
demographics.living_in_community_percent(params=params)

89.06

### Number of Ppts Indicated to Attend the Day Center
Count of distinct ppts from the center_days table enrolled during the period and the days column is not equal to 'PRN'.

In [79]:
demographics.attending_day_center(params=params)

229

### Percent of Ppts Indicated to Attend the Day Center
`attending_day_center` divided by `census_during_period`.

In [80]:
demographics.percent_attending_dc(params=params)

71.56

## Incidents
There are 5 incidents tables;
+ falls
+ med_errors
+ wounds
+ burns
+ infections

Some methods in the Incident class require an incident_table parameter to specify which type of incident the statistic is being calculated for.

In [81]:
incidents = pu.Incidents()

### Incidents per 100 Member Months (Falls/Med Errors/Wounds/Burns/Infections) 
Count of incidents with a date_time_occured date during the period divided by the sum of the first of the month census for each month during the period multiplied by 100.

In [82]:
incidents.incident_per_100MM(params=params, incident_table="falls")

13.08

In [83]:
incidents.incident_per_100MM(params=params, incident_table="med_errors")

0.54

### Number of Incidents (Falls/Med Errors/Wounds/Burns/Infections) 
Count of incidents with a date_time_occured date during the period.

In [84]:
incidents.total_incidents(params=params, incident_table="falls")

121

In [85]:
incidents.total_incidents(params=incidents.last_quarter(), incident_table="falls")

116

### Number of Ppts with more than 1 Incident in Period (Falls/Med Errors/Wounds/Burns/Infections) 
Count of ppts with more than 1 incident during period.

In [86]:
incidents.num_of_incident_repeaters(params=params, incident_table="falls")

23

### Number of Incidents by Ppts with more than 1 Incident in Period (Falls/Med Errors/Wounds/Burns/Infections) 
Sum of the count of incidents attributed to ppts with more than 1 incident in the period.

In [87]:
incidents.incidents_by_repeaters(params=params, incident_table="falls")

60

### Number of Unique Ppts with an Incident in Period (Falls/Med Errors/Wounds/Burns/Infections) 
Count of distinct ppts with an incident in the period.

In [88]:
incidents.ppts_w_incident(params=params, incident_table="falls")

84

### Percent of Incidents Attributed to Ppts with more than 1 Incident in the Period 
`incidents_by_repeaters` divided by `total_incidents`.

In [89]:
incidents.percent_by_repeaters(params=params, incident_table="falls")

49.59

### Rate of Ppts with more than 1 Incident of Ppts with an Incident
`num_of_incident_repeaters` divided by `ppts_w_incident`.

In [90]:
incidents.repeat_ppts_rate(params=params, incident_table="falls")

27.38

### Average Number of Incidents of Ppts with an Incident in the Period
Average number of incidents for participants who have had an incident in the period.

In [91]:
incidents.incident_avg_value(params=params, incident_table="falls")

1.4404761904761905

### Ppts with more Incidents than the Average During the Period
Ppts with more incidents during the period than the calculated average during the period.

In [92]:
incidents.ppts_above_avg(params=params, incident_table="falls")

23

### Rate of Ppts with an Above Average Incident Count out of Ppts with an Incident
`ppts_above_avg` divided by `ppts_w_incident`.

In [93]:
incidents.percent_of_ppts_over_avg(params=params, incident_table="falls")

27.38

### List of Ppts with More Than 1 Incident (Member Id, Incident Count)
Creates a list of member_id and incident count tuples for ppts with more than 1 incident in the period.

In [94]:
incidents.ppts_w_multiple_incidents(params=params, incident_table="falls")

[(100121, 3),
 (100263, 2),
 (100408, 2),
 (100422, 2),
 (100454, 2),
 (100541, 5),
 (100597, 2),
 (100605, 5),
 (100611, 2),
 (100751, 3),
 (100754, 4),
 (100757, 2),
 (100814, 2),
 (100815, 2),
 (100951, 2),
 (1000832, 2),
 (1000851, 2),
 (2000959, 2),
 (2000974, 3),
 (2001010, 2),
 (2001013, 3),
 (2001025, 4),
 (2001044, 2)]

### Adjusted Incident Count (Falls/Med Errors/Wounds/Burns/Infections)
Count of incidents during period minus any incidents by ppts with an incidents count greater than the mean plus 3 standard deviations.

In [95]:
incidents.adjusted_incident_count(params=params, incident_table="falls")

111

### Percent of Ppts without an Incident Ever
Count of ppts enrolled during period who do not have an incident divided by `census_during_period`.

In [96]:
incidents.percent_without_incident_overall(params=params, incident_table="falls")

32.19

### Percent of Ppts without an Incident in the Period
Count of ppts enrolled during period who do not have an incident during the period divided by `census_during_period`.

In [97]:
incidents.percent_without_incident_in_period(params=params, incident_table="falls")

73.75

### Number of Wounds Above Stage 1
Count of wounds with a date_time_occured during the period and an ulcer stage of Stage 2, Stage 3, Stage 4, or Unstageable.

In [98]:
incidents.wounds_above_stage1(params=params)

7

### Number of Wounds that are Unstageable
Count of wounds with a date_time_occured during the period and an ulcer stage of Unstageable.

In [99]:
incidents.unstageable_wounds_count(params=params)

1

### Number of Pressure Ulcers
Count of wounds with a date_time_occured during the period and a wound_type of Pressure Ulcer.

In [100]:
incidents.pressure_ulcer_count(params=params)

10

### Pressure Ulcers per 100 Member Months
`pressure_ulcer_count` divided by `member_months` multiplied by 100.

In [101]:
incidents.pressure_ulcer_per_100(params=params)

1.08

### Average Wound Healing Time
Average of the difference between the date_healed and the date_time_occured for wounds with a date_healed during the period.

In [102]:
incidents.avg_wound_healing_time(params=params)

45.97

### Number of UTIs
Counts of infections during the period with an infection_type of UTI, URI, or Sepsis-Urinary.

In [103]:
incidents.uti_count(params=params)

44

### UTIs per 100 Member Months
`uti_count` divided by `member_months` multiplied by 100.

In [104]:
incidents.uti_per_100(params=params)

4.76

### Number of Sepsis Infections
Count of infections during the period where 'sepsis' is in the infection_type.

In [105]:
incidents.sepsis_count(params=params)

0

### Sepsis Infection per 100 Member Months
`sepsis_count` divided by `member_months` multiplied by 100.

In [106]:
incidents.sepsis_per_100(params=params)

0.0

### Number of Burns with Degree 3rd Degree of Higher
Count of burns during period with a burn_degree of Third or Fourth.

In [107]:
incidents.third_degree_burn_count(params=params)

0

### Count of Burns Grouped by Degree
Count of burns during period grouped by degree.

In [108]:
incidents.burn_degree_counts(params=params)

[('Second', 2)]

### Number of Incidents with Major Harm or Death (Falls/Med Errors)
Count of incidents during period where the severity is equal to Major Harm or Death.

In [109]:
incidents.major_harm_or_death_count(params=params, incident_table="falls")

4

### Count of Med Error Responsibility by Type
The sum of each of the responsibility columns (responsibility_pharmacy, responsibility _clinic, responsibility _home_care, responsibility _facility) in the med_errors table for med_errors occurring during the period. Returns a pandas dataframe with the columns `responsibility` and `count`.

In [110]:
incidents.med_errors_responsibility_counts(params=params)

,responsibility,count
0,Pharmacy,2
1,Clinic,1
2,Home Care,0
3,Facility,0


### Most Common Med Error Responsibility Type
Creates the `med_errors_responsibility_counts` table and returns the first row as a tuple.

In [111]:
incidents.most_common_med_errors_responsibility(params=params)

('Pharmacy', 2)

### Number of Burns with an RN Assessment as a Follow Up
Sum of the assessment_rn column from burns for burns occurring during the period.

In [112]:
incidents.rn_assessment_following_burn_count(params=params)

2.0

### Number of Med Errors Related to a High Risk Medication
Finds descriptions that contain a high risk medication.

In [113]:
incidents.high_risk_med_error_count(params=params)

0

### Percent of Incidents Resulting in Major Harm or Death (Falls/Med Errors)
`major_harm_or_death_count` divided by `num_incident` multiplied by 100.

In [114]:
incidents.major_harm_percent(params=params, incident_table="falls")

3.31

### Adjusted Incidents per 100 Member Months (Falls/Med Errors/Wounds/Burns/Infections)
`adjusted_incident_count` divided by `member_months` 

In [115]:
incidents.adjusted_per_100MM(params=params, incident_table="falls")

12.0

### Percent of Wound that are Unstageable
`unstageable_wounds_count` divided by `total_incidents` (with parameter of "wounds")

In [116]:
incidents.unstageable_wound_percent(params=params)

0.03

### Percent of Burns with Degree 3rd Degree of Higher
`third_degree_burn_count` divided by `total_incidents` (with parameter of "burns")

In [117]:
incidents.third_degree_burn_rate(params=params)

0.0

### Percent of Burns that have an RN Assessment as Follow Up
`rn_assessment_following_burn_count` divided by `total_incidents` (with parameter of "burns")

In [118]:
incidents.rn_assessment_following_burn_percent(params=params)

100.0

## Utilization

There are two utilization tables (inpatient and er_only). There are also 6 views of the inpatient table;
+ acute
+ psych
+ nursing_home
+ skilled
+ respite
+ custodial

Some methods in the Utilization class require a utilization_table parameter to specify which type of utilization the statistic is being calculated for.

In [119]:
utilization = pu.Utilization()

### Admissions (Acute/Psych/Nursing Home/Custodial/Respite/Skilled)
Count of admissions with an admission_date in the period.

In [120]:
utilization.admissions_count(params=params, utilization_table="acute")

88

### Discharges (Acute/Psych/Nursing Home/Custodial/Respite/Skilled)
Count of admissions with a discharge_date in the period.

In [121]:
utilization.discharges_count(params=params, utilization_table="acute")

90

### Average Length of Stay  (Acute/Psych/Nursing Home/Custodial/Respite/Skilled)
Average los (length of stay) for admissions with a discharge_date in the period.

In [122]:
utilization.alos(params=params, utilization_table="acute")

4.5

### ER to Acute Admission Rate
Sum of the er column from the acute view divided by the sum of that value and the count of er_only admissions during the period.

In [123]:
utilization.er_to_inp_rate(params=params)

0.69

### Average Length of Stay for Acute Admissions from the ER
Average los for admissions in the acute with an er value of 1 and an admission_date during the period.

In [124]:
utilization.alos_for_er_admissions(params=params)

4.4

### Length of Stay per 100 Member Months
Sum of los for admissions with an admission_date during the period divided by the sum of the first of the month census for each month during the period multiplied by 100.

In [125]:
utilization.los_per_100mm(params=params, utilization_table="acute")

41.08

### Number of 30 Day Readmits
Count of admissions during the period with a day_since_last_admission value below 30.

In [126]:
utilization.readmits_30day(params=params, utilization_table="acute")

25

### 30 Day Readmit Rate
`readmits_30_day` divided by `admissions_count`

In [127]:
utilization.readmits_30day_rate(params=params, utilization_table="acute")

0.28

### Number of Ppts in a Utilization Type During the Period (Acute/Psych/Nursing Home/Custodial/Respite/Skilled)
Count of admissions with a discharge date that is null or greater than the start date and an admission_date less than the end date.

In [128]:
utilization.ppts_in_utl(params=params, utilization_table="acute")

92

### Ppts in Utilization Type per 100 Member Months  (Acute/Psych/Nursing Home/Custodial/Respite/Skilled)
`ppts_in_utl` divided by `member_months` multiplied by 100. 

In [129]:
utilization.ppts_in_utl_per_100MM(params=params, utilization_table="acute")

9.95

### Percent of Ppts in a Utilization Type
`ppts_in_utl` divided by `census_during_period`.

In [130]:
utilization.ppts_in_utl_percent(params=params, utilization_table="acute")

28.75

### Discharge 30 Days or Less before Death
Checks for an admission with a discharge date between the decease_date and the date 30 days prior, returns 1 if an admission exists, 0 otherwise.

In [131]:
utilization.check_for_admission_30days_before_death(member_id=100122, deceased_date="2007-03-26")

0

### Utilization Related to Condition or Diagnosis
Searches through the acute, er_only, admitting_claims, and claims_detail tables for any dx where the condition or condition_abr appear. Returns a pandas dataframe.

In [132]:
utilization.utilization_related_to_condition(params=params,
                                             condition="Chronic Obstructive Pulmonary Disease",
                                             condition_abr="COPD")

,member_id,admission_date,discharge_date,los,facility,discharge_disposition,admission_scheduled,admit_reason,diagnosis,readmit,...,principle_diag_desc,diagnosis68_desc,diagnosis69_desc,diagnosis70_desc,diagnosis71_desc,diagnosis72_desc,diagnosis73_desc,diagnosis74_desc,diagnosis75_desc,visit_type
0,2000906,2019-01-03,2019-01-09,6.0,Roger Williams Medical Center,Home with home health nursing or rehabilitation,Emergent Unscheduled,None,None,No,...,Secondary malignant neoplasm of mediastinum,Myocardial infarction type 2,Malignant neoplasm of unspecified part of unsp...,Chronic obstructive pulmonary disease with (ac...,Unspecified hydronephrosis,Hemiplegia and hemiparesis following cerebral ...,Periapical abscess without sinus,Acute respiratory distress,Essential (primary) hypertension,inpatient
27,100785,2019-01-07,2019-01-11,4.0,Roger Williams Medical Center,Home with home health nursing or rehabilitation,Emergent Unscheduled,COPD exac,None,No,...,Unspecified atrial fibrillation,"Pneumonia, unspecified organism",Chronic obstructive pulmonary disease with acu...,Chronic obstructive pulmonary disease with (ac...,Atelectasis,Chronic diastolic (congestive) heart failure,Hemiplegia and hemiparesis following unspecifi...,"Other epilepsy, not intractable, without statu...","Urinary tract infection, site not specified",inpatient
28,100785,2019-01-16,2019-01-20,4.0,Roger Williams Medical Center,Deceased,Emergent Unscheduled,CVA/embolism,None,Yes,...,Unspecified atrial fibrillation,"Pneumonia, unspecified organism",Chronic obstructive pulmonary disease with acu...,Chronic obstructive pulmonary disease with (ac...,Atelectasis,Chronic diastolic (congestive) heart failure,Hemiplegia and hemiparesis following unspecifi...,"Other epilepsy, not intractable, without statu...","Urinary tract infection, site not specified",inpatient
57,1000822,2019-01-22,2019-01-26,4.0,Kent County Memorial Hospital,Home with home health nursing or rehabilitation,Emergent Unscheduled,CHF,None,No,...,Hypertensive heart disease with heart failure,Chronic obstructive pulmonary disease with (ac...,"Acute kidney failure, unspecified",Acute on chronic diastolic (congestive) heart ...,Atherosclerotic heart disease of native corona...,Presence of aortocoronary bypass graft,Personal history of transient ischemic attack ...,Nonrheumatic mitral (valve) insufficiency,Presence of automatic (implantable) cardiac de...,inpatient
67,100811,2019-01-10,2019-01-14,4.0,Roger Williams Medical Center,Nursing home or rehabilitation facility,Emergent Unscheduled,chest pain,None,Yes,...,Non-ST elevation (NSTEMI) myocardial infarction,Obstructive sleep apnea (adult) (pediatric),Atherosclerotic heart disease of native corona...,Paroxysmal atrial fibrillation,Hypertensive heart disease with heart failure,"Heart failure, unspecified",Hypokalemia,Type 2 diabetes mellitus with diabetic neuropa...,"Chronic obstructive pulmonary disease, unspeci...",inpatient
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946,100958,2019-01-01,NaN,NaN,Rhode Island Hospital,NaN,NaN,NaN,None,NaN,...,"Hemorrhage due to vascular prosthetic devices,...","Vomiting, unspecified","Hypothyroidism, unspecified",Hypertensive chronic kidney disease with stage...,"Chronic obstructive pulmonary disease, unspeci...","Chronic kidney disease, unspecified","Pure hypercholesterolemia, unspecified","Nicotine dependence, cigarettes, uncomplicated",Other specified postprocedural states,er_only
954,100422,2019-03-01,NaN,NaN,Roger Williams Medical Center,NaN,NaN,NaN,None,NaN,...,Dehydration,Syncope and collapse,Influenza due to unidentified influenza virus ...,Hypertensive heart disease with heart failure,"Heart failure, unspecified","Chronic obstructive pulmonary disease, unspeci...","Combined rheumatic disorders of mitral, aortic...","Rheumatoid arthritis, unspecified",Hypokalemia,er_only
966,1000851,2019-02-04,NaN,NaN,South County Hospital,NaN,NaN,NaN,None,NaN,...,"Alcohol dependence with intox

### LOS Over X During Period DataFrame
Finds admissions with a discharge_date during the period and a los values above the provided *x*.

In [133]:
utilization.los_over_x_df(params=params, x=7, utilization_table="acute").head()

,member_id,admission_date,discharge_date,los,facility,discharge_disposition,admission_scheduled,admit_reason,diagnosis,readmit,admission_type,w_six_months,dow,er,days_since_last_admission,visit_id
0,100541,2018-12-31,2019-01-09,9,Rhode Island Hospital,Nursing home or rehabilitation facility,Emergent Unscheduled,AKI/Anemia,None,Yes,Acute Hospital,0,Monday,1,6.0,20281772
1,100811,2018-12-27,2019-01-09,13,Rhode Island Hospital,Nursing home or rehabilitation facility,Emergent Unscheduled,PNA,None,Yes,Acute Hospital,0,Thursday,1,0.0,20282038
2,100348,2019-02-06,2019-02-14,8,Roger Williams Medical Center,Nursing home or rehabilitation facility,Emergent Unscheduled,"influenza, PNA",None,No,Acute Hospital,0,Wednesday,1,62.0,20290554
3,100454,2019-01-15,2019-01-22,7,Rhode Island Hospital,Nursing home or rehabilitation facility,Emergent Unscheduled,Metapneumovirus,None,No,Acute Hospital,0,Tuesday,1,29.0,20290569
4,100540,2019-01-31,2019-02-14,14,Rhode Island Hospital,Nursing home or rehabilitation facility,Emergent Unscheduled,urosepsis/hypotension,None,Yes,Acute Hospital,0,Thursday,1,27.0,20290671


### Number of Admissions with a LOS Over X During Period
Count of admissions with a discharge_date during the period and a los values above the provided *x*.

In [134]:
utilization.los_over_x_count(params=params, x=7, utilization_table="acute")

17

### Days Over X During Period DataFrame
Days are calculated by subtracting the admission date from either the discharge date or the end date. This function then returns the admissions during the period with days above *x*.

In [135]:
utilization.days_over_x_df(params=params, x=7, utilization_table="acute").head()

,member_id,admission_date,discharge_date,facility,days
0,100541,2018-12-31,2019-01-09,Rhode Island Hospital,9.0
1,100811,2018-12-27,2019-01-09,Rhode Island Hospital,13.0
2,100348,2019-02-06,2019-02-14,Roger Williams Medical Center,8.0
3,100454,2019-01-15,2019-01-22,Rhode Island Hospital,7.0
4,100540,2019-01-31,2019-02-14,Rhode Island Hospital,14.0


### Number of Days for Utilization Type in Period
First the days for admissions that occur during the params is calculated. Next the days for admissions occurring before the period are calculated using the start date of the period as the admissions date. The sum of these two values are returned as the total utilization days during a given period.

In [136]:
utilization.utilization_days(params=params, utilization_table="acute")

451.0

### Days for Utilization Type per 100 Member Months
`utilization_days` divided by `member_months`.

In [137]:
utilization.days_per_100MM(params=params, utilization_table="acute")

48.76

### Admit Reasons by LOS
Finds admissions with a discharge_date during the period, orders them by los and then returns the *top_x* number of rows from that query as a pandas dataframe.

In [138]:
utilization.top_admit_reason_by_los(params=params, top_x=10)

,admit_reason,los,facility,admission_date,visit_id
0,URI,26,Rhode Island Hospital,2019-02-01,20290770
1,"anemia, GI bleed",15,Kent County Memorial Hospital,2019-01-22,22191105
2,urosepsis/hypotension,14,Rhode Island Hospital,2019-01-31,20290671
3,PNA,13,Rhode Island Hospital,2018-12-27,20282038
4,"COPD ex,RSV,PNA",11,The Miriam Hospital,2019-01-16,20290963
5,aggression and anxiety,10,Rhode Island Hospital,2019-01-20,20290725
6,AKI/Anemia,9,Rhode Island Hospital,2018-12-31,20281772
7,"influenza, PNA",8,Roger Williams Medical Center,2019-02-06,20290554
8,FAll/ LOC/Severe Anemia,8,Landmark Medical Center,2019-01-19,22191138
9,Metapneumovirus,7,Rhode Island Hospital,2019-01-15,20290569


### Top 10 ER Users
Counts the numbers of visits with an admission_date during the period grouped by participants and returns the top 10 rows as a pandas dataframe.

In [139]:
utilization.top_10_er_users(params=params)

,member_id,last,first,visits
0,2001012,Alvarez,Raquel,6
1,100814,Pasela,John,3
2,2001025,Poole,Chester,2
3,1000851,McGowan,Charles,2
4,1000824,Barclay,Elizabeth,2
5,100701,Gonsalez,Efrain,2
6,2001045,Vargas,Marcelino,1
7,2001020,Horton,Walter,1
8,2001018,Moisan,Rose,1
9,2000972,Flores,Marie,1


### 30 Day Readmission DataFrame
Returns a pandas dataframe of any admissions with an admission_date during the period and a day_since_last_admission value less than or equal to 30.

In [140]:
utilization.admissions_30day_readmit_df(params=params, utilization_table="acute").head()

,member_id,admission_date,discharge_date,los,facility,discharge_disposition,admission_scheduled,admit_reason,diagnosis,readmit,admission_type,w_six_months,dow,er,days_since_last_admission,visit_id
0,100303,2019-01-25,2019-01-27,2,Rhode Island Hospital,Home with home health nursing or rehabilitation,Emergent Unscheduled,FLU,None,No,Acute Hospital,0,Friday,1,30.0,20290428
1,100395,2019-01-08,2019-01-10,2,Landmark Medical Center,Home with home health nursing or rehabilitation,Emergent Unscheduled,COPD Exacerbation,None,Yes,Acute Hospital,0,Tuesday,1,11.0,20290503
2,100395,2019-01-12,2019-01-17,5,Landmark Medical Center,Nursing home or rehabilitation facility,None,None,None,Yes,Acute Hospital,0,Saturday,1,2.0,20290507
3,100454,2019-01-15,2019-01-22,7,Rhode Island Hospital,Nursing home or rehabilitation facility,Emergent Unscheduled,Metapneumovirus,None,No,Acute Hospital,0,Tuesday,1,29.0,20290569
4,100454,2019-02-13,2019-02-15,2,Rhode Island Hospital,Assisted Living Facility,Emergent Unscheduled,chest pain,None,Yes,Acute Hospital,0,Wednesday,1,22.0,20290667


### Admissions Resulting in a 30 Day Readmit DataFrame
Finds the admission prior to any admissions in the `acute_admissions_30day_readmit_df`. Returns these admissions as a pandas dataframe.

In [141]:
utilization.admissions_resulting_in_30day_df(params=params, utilization_table="acute").head()

,member_id,admission_date,discharge_date,facility,los,admit_reason
0,100303,2018-12-21,2018-12-26,The Miriam Hospital,5,CHF exac
0,100395,2018-12-27,2018-12-28,Landmark Medical Center,1,None
0,100395,2019-01-08,2019-01-10,Landmark Medical Center,2,COPD Exacerbation
0,100454,2018-12-07,2018-12-17,Rhode Island Hospital,10,multifocal pna & supratherapeutic INR
0,100454,2019-01-15,2019-01-22,Rhode Island Hospital,7,Metapneumovirus


### Unique Ppts with an Admission (Acute/Psych/Nursing Home/Custodial/Respite/Skilled)
Returns a count of the distinct ppts with an admission during the period.

In [142]:
utilization.unique_admissions_count(params=params, utilization_table="acute")

65

### Number of Weekend Admissions
Returns a count of the admissions during the period with a dow value of 'Saturday' or 'Sunday'

In [143]:
utilization.weekend_admissions_count(params=params, utilization_table="acute")

16

### Percent of Admissions that Occur on the Weekend
`weekend_admissions_count` divided by `admissions_count` multiplied by 100.

In [144]:
utilization.weekend_admission_percent(params=params, utilization_table="acute")

18.18

### Admissions per 100 Member Months
Sum of the count of admissions with an admission_date during the period divided by the sum of the first of the month census for each month during the period multiplied by 100.

In [145]:
utilization.admissions_per_100MM(params=params, utilization_table="acute")

9.51

### Number of Nursing Facility Discharges to Custodial or Acute Hospital 
Finds all nursing_home admissions or discharges during the period. Merges any discharges that have a disposition of "Nursing home or rehabilitation facility" with all nursing_home admissions on the discharge_date matching the admission_date. Filter to only include those discharges with a non-null admission_date and a new admit_reason of "custodial" where a previous admit_reason was not custodial and count the discharges in this filtered group.

It then sums up any discharges to "Acute care hospital or psychiatric facility".

These two values are added together and returned. 

In [146]:
utilization.nf_discharged_to_higher_loc(params=params)

33

### Percent of Nursing Facility Discharges that are to Custodial or Acute Hospital
`nf_discharged_to_higher_loc` divided by `discharges_count` for the nursing_home table multiplied by 100.

In [147]:
utilization.percent_nf_discharged_to_higher_loc(params=params)

51.56

## Quality

In [148]:
quality = pu.Quality()

### Ppts Who Need Pneumococcal 23 Vaccination DataFrame
Finds any ppts over 65 and enrolled during the period who does not have a record of receiving Pneumococcal 23. Returns a pandas dataframe of these ppts.

In [149]:
quality.dataframe_query("SELECT * FROM ppts").columns

Index(['member_id', 'last', 'first'], dtype='object')

In [150]:
quality.need_pneumo_23_df(params=params).head()

,member_id,last,first,team,enrollment_date,age
0,100250,Charron,Roger,South,2008-04-01,70.872005
1,100303,Simmonds,Alejandro,East,2008-11-01,82.910335
2,100307,Cabral,Maria,East,2008-10-01,73.149897
3,100434,Stanhope,Marilynn,South,2010-08-01,75.345654
4,100481,Sical,Zoila,Central,2011-06-01,79.441478


### Ppts Who Need PCV 13 Vaccination DataFrame
Find any ppts over 65 and enrolled during the period who does not have a record of receiving PCV 13. Returns a pandas dataframe of these ppts.

In [151]:
quality.need_pcv_13_df(params=params).head()

,member_id,last,first,team,enrollment_date,age
0,100113,Ricci,Mary,North,2006-03-01,93.837098
1,100202,Allen,Janet,North,2007-05-01,85.371663
2,100232,Markarian,Azniv,East,2007-11-01,91.403149
3,100247,Segerson,Anne,South,2008-03-01,67.216975
4,100250,Charron,Roger,South,2008-04-01,70.872005


### Ppts Who Need ONLY Pneumococcal 23 Vaccination DataFrame

Finds any ppts over 65 and enrolled during the period who does not have a record of receiving Pneumococcal 23 and has a record of receiving PCV 13. Returns a pandas dataframe of these ppts.

In [152]:
quality.need_pneumo_23_only_df(params=params).head()

,member_id,last,first,team,enrollment_date,age
0,100303,Simmonds,Alejandro,East,2008-11-01,82.910335
1,100756,Stephens,Martha,Central,2015-05-01,79.329227
2,100756,Stephens,Martha,Central,2015-05-01,79.329227
3,100804,Houle,Normand,North,2016-04-01,87.137577
4,1000850,Sommerville,Raymond,East,2017-10-01,77.749487


### Ppts Who Need ONLY PCV 13 Vaccination DataFrame
Finds any ppts over 65 and enrolled during the period who does not have a record of receiving PCV 13 and has a record of receiving Pneumococcal 23. Returns a pandas dataframe of these ppts.

In [153]:
quality.need_pcv_13_only_df(params=params).head()

,member_id,last,first,team,enrollment_date,age
0,100113,Ricci,Mary,North,2006-03-01,93.837098
1,100131,Killeen,Linda,North,2006-06-01,70.379192
2,100131,Killeen,Linda,West,2006-06-01,70.379192
3,100141,Krong,Sida,East,2006-07-01,80.826831
4,100141,Krong,Sida,West,2006-07-01,80.826831


### Ppts Who Need BOTH Pneumococcal 23 & PCV 13 Vaccination DataFrame

Returns ppts who do not have either PCV 13 or Pneumococcal 23. If include_refused is True, will include participants who have previously refused the vaccination as still needing it, otherwise they are removed.

In [154]:
quality.need_both_pneumo_vaccs_df(params=params).head()

,member_id,last,first,team,enrollment_date,age
0,100250,Charron,Roger,South,2008-04-01,70.872005
1,100307,Cabral,Maria,East,2008-10-01,73.149897
2,100434,Stanhope,Marilynn,South,2010-08-01,75.345654
3,100481,Sical,Zoila,Central,2011-06-01,79.441478
4,100504,Rodriguez,Diana,East,2011-11-01,85.234771


### Number of Ppts with Pneumococcal Vaccination
Count of all enrolled ppts over the age of 65 who have at least one of the pneumo vaccinations.

In [155]:
quality.has_pneumo_vacc_count(params=params)

233

### Number of Ppts who Refused Pneumococcal Vaccination

Count of all enrolled ppts over the age of 65 who have refused the vaccination and not received at least one of the pneumo vaccinations.

In [156]:
quality.refused_pneumo_vacc_count(params=params)

26

### Pneumococcal Vaccination Rate
`has_pneumo_vacc_count` plus `refused_pneumo_vacc_count` divided by `age_above_65`.

In [157]:
quality.pneumo_rate(params=params)

0.94

### Ppts who Refused Pneumococcal Vaccination DataFrame

Dataframe of all enrolled ppts over the age of 65 who have refused the vaccination and not received at least one of the pneumo vaccinations.

In [158]:
quality.refused_pneumo_vacc_df(params=params).head()

,member_id,vacc_series,date_administered,last,first,team,enrollment_date,age
0,100620,Pneumococcal 23,2016-03-31,Burke,Madelyn,East,2013-06-01,86.907598
1,100307,PCV 13,2018-10-10,Cabral,Maria,East,2008-10-01,73.149897
2,100307,Pneumococcal 23,2017-03-16,Cabral,Maria,East,2008-10-01,73.149897
3,100250,Pneumococcal 23,2010-07-27,Charron,Roger,South,2008-04-01,70.872005
4,2000936,PCV 13,2018-10-05,Charron,Virginia,East,2018-06-01,82.179329


### Ppts with Pneumococcal Vaccination DataFrame
Dataframe of all enrolled ppts over the age of 65 who have at least one of the pneumo vaccinations.

In [159]:
quality.has_pneumo_vacc_df(params=params).head()

,member_id,vacc_series,date_administered,last,first,team,enrollment_date,age
0,100746,PCV 13,2018-02-22,Acosta,Gloria,South,2015-02-01,84.580424
1,100746,Pneumococcal 23,2015-01-01,Acosta,Gloria,South,2015-02-01,84.580424
2,100420,Pneumococcal 23,2012-05-15,Alamo-Vazquez,Tomasa,North,2010-05-01,94.026010
3,2000855,Pneumococcal 23,2017-11-30,Alba,Felicia,South,2017-10-01,83.564682
4,100564,Pneumococcal 23,2010-09-10,Alcindor,Marie,Central,2012-09-01,93.385352


### Influenza Related Calculations
If the start date of the period is before April the start of the flu season is considered to be September 1st of the previous year. Other wise it will be the 1st of September in the same year as the parameter start.

This means the period of April through August will always have no results.

### Ppts Who Need to be Offered the Influzenza Vaccination DataFrame
Returns a dataframe of ppts enrolled during the period who do not have an influenza action (administer or refused) during the related flu season.

In [160]:
quality.need_influenza_vacc_df(params=params).head()

,member_id,last,first,team,enrollment_date
0,100232,Markarian,Azniv,East,2007-11-01
1,100261,Hernandez,Ismael,South,2008-05-01
2,100540,Marcucci,Santos,Central,2012-05-01
3,100564,Alcindor,Marie,Central,2012-09-01
4,100623,Morel,Aurelia,Central,2013-06-01


### Number of Ppts with Influzenza Vaccination
Count of ppts who have an influenza vaccination (dose_status=1) during the related flu season.

In [161]:
quality.has_influ_vacc_count(params=params)

233

### Ppts with Influzenza Vaccination DataFrame
Returns a dataframe of ppts enrolled during the period who have an influenza vaccination (dose_status=1) during the related flu season.

In [162]:
quality.has_influ_vacc_df(params=params).head()

,member_id,vacc_series,date_administered,last,first,team,enrollment_date
0,100278,Influenza_,2019-09-26,Fairweather,Danette,East,2008-07-01
1,100278,Influenza_,2019-09-26,Fairweather,Danette,South,2008-07-01
2,2001027,Influenza_,2019-09-26,Recinos,Maria,East,2019-02-01
3,2000952,Influenza_,2019-10-02,Curley,Cynthia,North,2018-07-01
4,2001002,Influenza_,2019-10-04,Eccleston,Douglas,South,2018-11-01


### Number of Ppts who Refused Influenza
Count of ppts who have refused the influenza vaccination (dose_status=0) during the related flu season and not subsequently received it.

In [163]:
quality.refused_influ_vacc_count(params=params)

41

### Ppts who Refused Influenza DataFrame
Returns a dataframe of ppts enrolled during the period who have refused an influenza vaccination during the related flu season.

In [164]:
quality.refused_influ_vacc_df(params=params).head()

,member_id,vacc_series,date_administered,last,first,team,enrollment_date
0,2000936,Influenza_,2019-09-26,Charron,Virginia,East,2018-06-01
1,2001020,Influenza_,2019-10-01,Horton,Walter,East,2019-01-01
2,2000972,Influenza_,2019-10-02,Flores,Marie,North,2018-09-01
3,100850,Influenza_,2019-09-27,Garcia De Mella,Juana,Central,2017-04-01
4,100751,Influenza_,2019-10-04,Urreta,Deborah,North,2015-04-01


### Influenza Vaccination Rate
The sum of ppts enrolled during the period who have either received or refused the influenza vaccination divided by the census during the period.

In [165]:
quality.influ_rate(params=params)

0.86

### Mortality Rate
The number of deaths during the period divided by the census during the period.

In [166]:
quality.mortality_rate(params=params)

10

### Mortality within 30 Days of Discharge Rate
The rate of deaths that occurred within 30 days of discharge from a hospital. This is deaths within 30 days of discharge divided by total deaths during the period.

In [167]:
quality.mortality_within_30days_of_discharge_rate(params=params)

0.2

### Percent of Discharges that Result in Death within 30 Days
The rate of discharges that result in death within 30 days. This is deaths within 30 days of discharge divided by total hospital discharges during the period.

In [168]:
quality.percent_of_discharges_with_mortality_in_30(params=params)

2.22

### Percent of Ppts without Acute Admission Since Enrollment
Percent of ppts who do not have a recorded acute hospital admissions.

In [169]:
quality.no_hosp_admission_since_enrollment(params=params)

39.69

### Percent of Ppts without Acute Admission in the Last Year
Percent of ppts who do not have a recorded acute hospital admissions within the year prior to the provided end date.

In [170]:
quality.no_hosp_admission_last_year(params=params)

46.25

### Average Days From Enrollment to First Custodial Admission or Average Days Until Nursing Facility Admission
The average of the difference between the first admission date to a custodial stay and the ppt enrollment date for ppts enrolled in the period.

In [171]:
quality.avg_days_until_nf_admission(params=params)

1624.67

## Team

Team functions take the functions listed in the modules above and create dataframe where each row is the team and the value of the function for that team.

In [172]:
team = pu.Team()

### Admissions by Team

In [173]:
team.admissions_by_team(params=params, utilization_table="acute")

,team,acute admissions
0,None,1
1,Central,14
2,East,31
3,North,19
4,South,23


### Utilization Days by Team

In [174]:
team.days_by_team(params=params, utilization_table="acute")

,team,acute days
0,None,5.0
1,Central,345.0
2,East,201.0
3,North,73.0
4,South,112.0


### Discharges by Team

In [175]:
team.discharges_by_team(params=params, utilization_table="acute")

,team,acute discharges
0,None,1
1,Central,17
2,East,30
3,North,19
4,South,23


### Average Length of Stay by Team

In [176]:
team.alos_for_discharges_by_team(params=params, utilization_table="acute")

,team,acute alos
0,None,5.00
1,Central,5.76
2,East,3.90
3,North,3.84
4,South,4.87


### 30 Day Readmissions by Team

In [177]:
team.readmits_by_team(params=params, utilization_table="acute")

,team,acute 30 day readmits
0,Central,3
1,East,14
2,North,5
3,South,3


### Average Age by Team

In [178]:
team.avg_age_by_team(params=params)

,team,age
0,None,58.44
1,Central,78.03
2,East,76.88
3,North,75.42
4,South,75.39


### Percent Non-English Primary Speakers by Team

In [179]:
team.percent_primary_non_english_by_team(params=params)

,team,percent non english
0,None,0.00
1,Central,48.81
2,East,52.81
3,North,27.27
4,South,25.00


### Average Years Enrolled in PACE by Team

In [180]:
team.avg_years_enrolled_by_team(params=params)

,team,avg_years_enrolled
0,None,0.41
1,Central,4.28
2,East,5.15
3,North,3.73
4,South,4.36


### Number of Ppts in Custodial Care by Team

In [181]:
team.ppts_in_custodial_by_team(params=params)

,team,ppts in custodial
0,Central,18
1,East,15
2,North,4
3,South,10


### Number of Ppts on Team

In [182]:
team.ppts_on_team(params=params)

,team,participants
0,None,1
1,Central,84
2,East,89
3,North,66
4,South,76


### Member Months during Period for Team

In [185]:
team.team_member_months(params=params)

,team,participants
0,None,3
1,Central,243
2,East,259
3,North,187
4,South,222


### Mortality Rate by Team

In [186]:
team.mortality_by_team(params=("2017-01-01", "2019-10-01"))

,team,mortality rate
0,None,NaN
1,Central,0.005758
2,East,0.004067
3,North,0.007161
4,South,0.006661


### Deaths by Team

In [187]:
team.mortality_by_team(params=params, total=True)

,team,deaths
0,Central,2
1,East,3
2,North,3
3,South,2


### Mortality within 30 Days of Discharge

In [188]:
team.mortality_within_30days_of_discharge_by_team_df(params=params)

,team,30day_death_discharge
0,South,1
1,East,0
2,Central,0
3,North,1


### Percent of Discharges with a Death within 30 Days by Team

In [189]:
team.percent_of_discharges_with_mortality_in_30_by_team(params=params)

,team,% of discharges with death within 30 days
0,South,4.347826
1,East,0.000000
2,Central,0.000000
3,North,5.263158


### Mortality within 30 Days of Discharge Rate by Team

In [190]:
team.mortality_within_30days_of_discharge_rate_by_team(params=params)

,team,death within 30 days of discharge date
0,South,0.500000
1,East,0.000000
2,Central,0.000000
3,North,0.333333


### Percent of Ppts with No Acute Admission Since Enrollment by Team

In [191]:
team.no_hosp_admission_since_enrollment_by_team(params=params)

,team,percent with no admissions since enrollment
0,Central,13.168724
1,East,15.830116
2,North,13.368984
3,South,12.162162
4,None,NaN


### Pressure Ulcer Rate by Team

In [192]:
team.pressure_ulcer_rate_by_team(params=params)

,team,Wound Rate
0,North,0.042781
1,None,NaN
2,Central,NaN
3,East,NaN
4,South,NaN


### Incidents by Team (Falls/Med Errors/Wounds/Burns/Infections)

In [193]:
team.total_incidents_by_team(params=params, incident_table="falls")

,team,falls
0,Central,31
1,East,30
2,North,27
3,South,33


### Incidents per Member by Team (Falls/Med Errors/Wounds/Burns/Infections)

In [194]:
team.incidents_per_member_by_team(params=params, incident_table="falls")

,team,falls per 100 Ppts
0,Central,12.757202
1,East,11.583012
2,North,14.438503
3,South,14.864865
4,None,NaN


### Number of Ppts with an Incident by Team (Falls/Med Errors/Wounds/Burns/Infections)

In [195]:
team.ppts_w_incident_by_team(params=params, incident_table="falls")

,team,individuals w/ falls
0,Central,19
1,East,27
2,North,19
3,South,19


### Create a Time/Value DataFrame for Plotting

In [196]:
team.loop_plot_team_df(team.ppts_on_team, params=params, freq="MS", col_suffix="_census")

,none_census,central_census,east_census,north_census,south_census,month
0,1,80,88,62,73,2019-01-01
1,1,81,87,62,74,2019-02-01
2,1,82,84,63,75,2019-03-01


### Create a Time/Value DataFrame for Plotting

In [197]:
team.loop_plot_team_df(team.ppts_w_incident_by_team,
                       params=params,
                       freq="QS",
                       additional_func_args = ["falls"],
                       col_suffix="_census")

,none_census,central_census,east_census,north_census,south_census,month
0,0,19.0,27.0,19.0,19.0,2019-01-01
1,0,16.0,23.0,16.0,16.0,2019-02-01
2,0,17.0,19.0,16.0,16.0,2019-03-01


## Agg

Defaults to the Agg SQLite database(V:\Databases\agg.db), contains a few tables with aggregate data by month or quarter. Mostly used for plotting on the dashboard

Add `_q` to any table below to return the data aggregated by quarter instead of month.

+ enrollment
+ demographics
+ falls
+ med_errors
+ wounds
+ burns
+ infections
+ utilization
+ quality
+ team_utl
+ team_info
+ team_incidents
+ center_enrollment

In [198]:
agg = pu.Agg()

In [199]:
agg.get_plot_df(table="enrollment", col="census", params=("2019-01-01", "2019-10-01"))

,month,census
0,2019-01-01,302
1,2019-02-01,303
2,2019-03-01,305
3,2019-04-01,311
4,2019-05-01,310
5,2019-06-01,313
6,2019-07-01,317
7,2019-08-01,320
8,2019-09-01,335
9,2019-10-01,335


In [200]:
agg.get_plot_df(table="enrollment_q", col="census", params=("2019-01-01", "2019-10-01"))

,month,census
0,2019-01-01,292
1,2019-04-01,301
2,2019-07-01,311
3,2019-10-01,339


In [201]:
agg.team_plot_df(table="team_info", col="ppts", params=("2019-01-01", "2019-10-01"))

,month,None,Central,East,North,South
0,2019-01-01,1.0,71.0,77.0,52.0,67.0
1,2019-02-01,1.0,75.0,78.0,54.0,68.0
2,2019-03-01,1.0,75.0,78.0,55.0,70.0
3,2019-04-01,1.0,76.0,79.0,60.0,71.0
4,2019-05-01,1.0,76.0,80.0,63.0,72.0
5,2019-06-01,1.0,77.0,80.0,66.0,74.0
6,2019-07-01,1.0,79.0,82.0,69.0,75.0
7,2019-08-01,1.0,80.0,83.0,71.0,76.0
8,2019-09-01,1.0,82.0,89.0,77.0,80.0
9,2019-10-01,1.0,84.0,90.0,78.0,82.0


In [202]:
agg.team_plot_df(table="team_utl_q", col="custodial_ppts", params=("2019-01-01", "2019-10-01"))

,month,None,Central,East,North,South
0,2019-01-01,0,14.0,9.0,3.0,10.0
1,2019-04-01,0,12.0,13.0,3.0,11.0
2,2019-07-01,0,16.0,16.0,2.0,18.0
3,2019-10-01,0,16.0,18.0,5.0,17.0


# WIP(Beta) Classes

## Participant

In [203]:
ppt = pu.Participant()

### Utilization Related to Ppt in Period

In [204]:
ppt.utilization(params=("2018-09-01", "2019-09-30"), utilization_table="acute", member_id=100811)

,member_id,admission_date,discharge_date,los,facility,discharge_disposition,admission_scheduled,admit_reason,diagnosis,readmit,admission_type,w_six_months,dow,er,days_since_last_admission,visit_id
0,100811,2018-10-14,2018-10-17,3,Rhode Island Hospital,Assisted Living Facility,Emergent Unscheduled,r/o MI,None,No,Acute Hospital,0,Sunday,1,254.0,20281825
1,100811,2018-10-18,2018-10-19,1,Rhode Island Hospital,Assisted Living Facility,Emergent Unscheduled,chest pain,None,Yes,Acute Hospital,0,Thursday,1,1.0,20281829
2,100811,2018-10-27,2018-10-28,1,Rhode Island Hospital,Assisted Living Facility,Emergent Unscheduled,rapid heartbeat,None,Yes,Acute Hospital,0,Saturday,1,8.0,20281838
3,100811,2018-12-22,2018-12-27,5,Rhode Island Hospital,Assisted Living Facility,Emergent Unscheduled,None,None,No,Acute Hospital,0,Saturday,1,55.0,20282033
4,100811,2018-12-27,2019-01-09,13,Rhode Island Hospital,Nursing home or rehabilitation facility,Emergent Unscheduled,PNA,None,Yes,Acute Hospital,0,Thursday,1,0.0,20282038
5,100811,2019-01-10,2019-01-14,4,Roger Williams Medical Center,Nursing home or rehabilitation facility,Emergent Unscheduled,chest pain,None,Yes,Acute Hospital,0,Thursday,1,1.0,20290921
6,100811,2019-01-25,2019-01-29,4,Roger Williams Medical Center,Nursing home or rehabilitation facility,Emergent Unscheduled,NSTEMI,None,Yes,Acute Hospital,0,Friday,1,11.0,20290936
7,100811,2019-02-24,2019-02-25,1,Kent County Memorial Hospital,Nursing home or rehabilitation facility,Emergent Unscheduled,None,None,Yes,Acute Hospital,0,Sunday,1,26.0,20291035
8,100811,2019-03-17,2019-03-18,1,Kent County Memorial Hospital,Nursing home or rehabilitation facility,Emergent Unscheduled,abd pain,None,Yes,Acute Hospital,0,Sunday,1,20.0,20291128


### Get Ppt Name from Member ID

In [205]:
ppt.name(member_id=100811)

[(' Frederick', 'Farland')]

## Center Classes
### CenterEnrollment & CenterDemographics

Working on completing these or moving to add a "center" parameter to all of the functions in the classes above.

In [206]:
center_enrollment = pu.CenterEnrollment()

In [207]:
center_enrollment.census_during_period(params=params, center="Providence")

259

In [208]:
enrolled_addresses, disenrolled_addresses = center_enrollment.address_mapping_df(center="Providence")
enrolled_addresses.head()

,name,full_address,lat,lon
0,M'liss Place-Sloan,"568 Child Street, Warren",41.732102,-71.254934
1,Linda Killeen,"25 Tobey St, Providence",41.819837,-71.436050
2,Sida Krong,"100 Atwells Ave, Providence",41.822834,-71.421123
3,Maria Alves,"100 Borden St, Providence",41.812769,-71.411593
4,Natividad Figueroa,"700 Westminster St, Providence",41.818269,-71.419381


In [209]:
center_demographics = pu.CenterDemographics()

In [210]:
print(center_demographics.percent_primary_non_english(params, center="Providence"))
print(center_demographics.percent_primary_non_english(params, center="Woonsocket"))
print(center_demographics.percent_primary_non_english(params, center="Westerly"))

43.63
29.23
0.0
